/home/dimweb/Desktop/deeppavlov/d_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from core.dataloaders.focus.focus_dataloader import FoCusTestDatasetV1
from core.inference.inference_scripts import (
    FocusPersonaExtractorV1,
    FocusKnowledgeKandidateExtractorV1,
    ResponseGeneratorV1,
    BartFocusTestDatasetV1,
)

test_dataset = FoCusTestDatasetV1(
    input_dataset_path="./datasets/FoCus/test_focus_public.json",
)

persona_model_name = "./results/microsoft/deberta-v3-small/checkpoint-87000/"
persona_extractor = FocusPersonaExtractorV1(
    model_name=persona_model_name,
)

knowledge_extractor = FocusKnowledgeKandidateExtractorV1()
response_extractor = ResponseGeneratorV1(
    model_name="./bart_base_2cx77pua",
)

bart_focus_test_dataset = BartFocusTestDatasetV1(
    initial_dataset=test_dataset,
    knowledge_kandidate_extractor=knowledge_extractor,
    focus_persona_extractor=persona_extractor,
    response_generator=response_extractor,
)

## create persona and knowledge prediction dataset

In [3]:
import json

with open("./predicts/bart_predict_full_dataset.json", "r") as f:
    predicts = json.load(f)

In [6]:
from core.inference.inference_scripts import make_submission 

# make_submission(
#     knowledge_persona_save_path=
#     predicts=
#     response_save_path=
# )